<a href="https://colab.research.google.com/github/arzuunr/100_day_challenge/blob/main/Aterimlerini%C3%87ekme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Hedef URL
url = "https://www.investopedia.com/terms-beginning-with-a-4769351"

# Headers
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd"
}

# Sayfayı çek
response = requests.get(url, headers=headers)

# Hata kontrolü
if response.status_code == 200:
    # HTML'yi parse et
    soup = BeautifulSoup(response.content, "html.parser")

    # Terimlerin bulunduğu HTML elementlerini bul (class'ı doğrulayın)
    terms = []
    for link in soup.find_all("a"):  # Önce tüm <a> etiketlerini dene
        term = link.text.strip()
        if term and len(term.split()) > 0:  # Boş veya anlamsız içerikleri filtrele
            terms.append(term)

    # Benzersiz terimleri al
    terms = list(dict.fromkeys(terms))  # Tekrarları kaldır

    # Terimleri bir DataFrame'e kaydet
    df = pd.DataFrame(terms, columns=["Term"])
    df.to_csv("investopedia_terms_a.csv", index=False)

    print(f"{len(terms)} terim çekildi. Örnek: {terms[:5]}")
else:
    print(f"Hata: {response.status_code} - {response.text}")

# Rate limiting için bekleme
time.sleep(2)

395 terim çekildi. Örnek: ['Skip to content', 'News', 'Markets', 'Companies', 'Earnings']


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url="https://www.odeabank.com.tr/hakkimizda/oblog/adan-zye-bankacilik-ve-finans-terimleri-sozlugu"

headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en;q=0.9"
 }

response=requests.get(url, headers=headers)
soup=BeautifulSoup(response.content, "html.parser")

terms=[]
for link in soup.find_all("a"):
  term=link.text.strip()
  if term and len(term.split()) > 0:  # Boş veya anlamsız içerikleri filtrele
            terms.append(term)

terms = list(dict.fromkeys(terms))
df = pd.DataFrame(terms, columns=["Term"])
df.to_csv("odeabank_terms_a.csv", index=False)

print(f"{len(terms)} terim çekildi. Örnek: {terms[:5]}")



97 terim çekildi. Örnek: ['İçeriğe atla', 'Alt bölüme atla', 'Ürün ve Hizmet Ücretleri', 'Hesaplama Araçları', 'Kampanyalar']


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = "https://www.odeabank.com.tr/hakkimizda/oblog/adan-zye-bankacilik-ve-finans-terimleri-sozlugu"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en;q=0.9"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

terms = []
# "A" başlıklı bölümü bul
a_section = soup.find("h2", string="A")  # "A" başlığını bul
if a_section:
    # Başlıktan sonraki terimleri al (örneğin, bir <ul> veya <div> içinde)
    term_container = a_section.find_next("ul") or a_section.find_next("div")  # Tahmini etiket
    if term_container:
        for link in term_container.find_all("a"):
            term = link.text.strip()
            if term and len(term.split()) > 0:
                terms.append(term)

terms = list(dict.fromkeys(terms))  # Tekrarları kaldır
df = pd.DataFrame(terms, columns=["Term"])
df.to_csv("odeabank_terms_a_only.csv", index=False)

print(f"{len(terms)} terim çekildi. Örnek: {terms[:5]}")

35 terim çekildi. Örnek: ['Hakkımızda', 'Banka Bilgileri', 'Kurumsal Yönetim', 'Kurumsal Kütüphane', 'İnsan Kaynakları - Odealist']
